In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow import keras
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import pandas as pd

In [2]:
def FFNN(input_shape,
         n_neurons,
         n_hidden,
         learning_rate):
    
    # neural network dynamic architecture
    model = Sequential()
    model.add(Dense(n_neurons, input_shape=(input_shape,), activation='relu'))
    for layer in range(n_hidden):
        model.add(Dense(n_neurons, activation="relu")) 
    model.add(Dense(1, activation='sigmoid'))

    # define stochastic gradient descent optimizer
    optimizer = SGD(lr=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [3]:
def create_model():
    
    # create model
    model = Sequential()
    model.add(Dense(12, input_shape=(8,), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer="SGD", metrics=['accuracy'])

    return model

In [4]:
# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)

# load dataset
dataset = pd.read_csv("diabetes.csv")

# split into input (X) and output (Y) variables
X = dataset.loc[:,dataset.columns[0:8]].to_numpy()
Y = dataset.loc[:,dataset.columns[8]].to_numpy()

In [5]:
# create model
model = keras.wrappers.scikit_learn.KerasClassifier(FFNN, verbose=0)

# define the grid search parameters
param_grid = dict(
    
    batch_size=[10, 20, 40, 60, 80, 100],
    epochs=[10, 50, 100],
    input_shape=[X.shape[1]],
    n_hidden=range(1, 2 + 1),
    n_neurons=[10],
    learning_rate=[0.001, 0.01, 0.1, 0.2, 0.3]
                 
)
grid = GridSearchCV(estimator=model,
                    param_grid=param_grid,
                    n_jobs=-1,
                    cv=3)
grid_result = grid.fit(X, Y)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.687500 using {'batch_size': 10, 'epochs': 50, 'input_shape': 8, 'learning_rate': 0.001, 'n_hidden': 1, 'n_neurons': 10}


In [6]:
# summarize results
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

0.641927 (0.010253) with: {'batch_size': 10, 'epochs': 10, 'input_shape': 8, 'learning_rate': 0.001, 'n_hidden': 1, 'n_neurons': 10}
0.597656 (0.011500) with: {'batch_size': 10, 'epochs': 10, 'input_shape': 8, 'learning_rate': 0.001, 'n_hidden': 2, 'n_neurons': 10}
0.610677 (0.064213) with: {'batch_size': 10, 'epochs': 10, 'input_shape': 8, 'learning_rate': 0.01, 'n_hidden': 1, 'n_neurons': 10}
0.639323 (0.021236) with: {'batch_size': 10, 'epochs': 10, 'input_shape': 8, 'learning_rate': 0.01, 'n_hidden': 2, 'n_neurons': 10}
0.651042 (0.024774) with: {'batch_size': 10, 'epochs': 10, 'input_shape': 8, 'learning_rate': 0.1, 'n_hidden': 1, 'n_neurons': 10}
0.651042 (0.024774) with: {'batch_size': 10, 'epochs': 10, 'input_shape': 8, 'learning_rate': 0.1, 'n_hidden': 2, 'n_neurons': 10}
0.651042 (0.024774) with: {'batch_size': 10, 'epochs': 10, 'input_shape': 8, 'learning_rate': 0.2, 'n_hidden': 1, 'n_neurons': 10}
0.651042 (0.024774) with: {'batch_size': 10, 'epochs': 10, 'input_shape': 8, 